In [1]:
require('nn')
require('hdf5')
require('optim')

f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
n_reduced = 1001
f:close()

window_size = X_train:size(2)
n_actual = nfeatures

In [2]:
n_actual


10002	


In [10]:

embedded_size = 100
D_h = 100

lookup1 = nn.LookupTable(nfeatures,embedded_size)
linear1 = nn.Linear(window_size*embedded_size, D_h)
tanh = nn.HardTanh()
linear2 = nn.Linear(D_h, n_actual)
h = nn.Sequential()
h:add(lookup1)
h:add(nn.Reshape(window_size*embedded_size, True))
h:add(linear1)
h:add(tanh)
h:add(linear2)

mse = nn.CrossEntropyCriterion()

In [14]:
w, dl_dw = h:getParameters()


feval = function(w_new)
   
    bsize= batch
    local idx = torch.randperm(X_train:size(1)):sub(1,bsize)

    x = torch.Tensor(bsize, X_train:size(2))

    
    --y = y:reshape(Y_train:size(1), 1)
    
    y_ = torch.Tensor(bsize, 1)

    for i=1,bsize do
        x[i] = X_train[idx[i]]
        y_[i] = Y_train[idx[i]]
    end
    
    y_ = y_:squeeze()
    
    local inputs = x
    local targets = y_
    -- reset gradients (gradients are always accumulated, to accommodate
    -- batch methods)
    dl_dw:zero()

    -- evaluate the loss function and its derivative with respect to x, given a mini batch
    local prediction = h:forward(inputs)
    local loss_w = mse:forward(prediction, targets)
    h:backward(inputs, mse:backward(prediction, targets))
    return loss_w, dl_dw
            
end

batch = 32

-- cycle on data
for i = 1,100 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,w)

    if i % 10 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] )
        -- print(sgd_params)
    end
end

loss for iteration 10 is 5.3462612690193	


loss for iteration 20 is 6.8115822413589	


loss for iteration 30 is 6.1787073569588	


loss for iteration 40 is 5.3995086481801	


loss for iteration 50 is 5.4829083572653	


loss for iteration 60 is 4.9841508303582	


loss for iteration 70 is 6.0303520975489	


loss for iteration 80 is 6.0063622168267	


loss for iteration 90 is 6.3071935072453	


loss for iteration 100 is 4.4564815836598	


In [17]:
function perplexity(input_set, output_set, kaggle)
    sum = 0
    for i = 1,input_set:size(1) do
        a = h:forward(input_set[i])
        b = nn.Index(1):forward{a,output_set[i]}
        distribution = nn.LogSoftMax():forward(b)
        answer = distribution[kaggle[i]]
        sum = sum + answer
    end

    return torch.exp(-sum/input_set:size(1))
end


In [16]:
perplexity()

13.72248283031	
